# Tiền Xử Lý Dữ Liệu

Notebook này thực hiện các bước làm sạch, tạo đặc trưng và chọn đặc trưng cho mô hình.


In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

DATA_PATH = "/Users/doananh/Documents/Documents - Doan’s MacBook Pro/đồ án DS/project2/data_motobikes.xlsx - Sheet1.csv"

raw = pd.read_csv(DATA_PATH)

# Làm sạch các cột số
for col in ["Giá", "Khoảng giá min", "Khoảng giá max"]:
    raw[col] = (raw[col]
                .astype(str)
                .str.lower()
                .str.replace("đ", "", regex=False)
                .str.replace("vnđ", "", regex=False)
                .str.replace("vnd", "", regex=False)
                .str.replace("triệu", "", regex=False)
                .str.replace("tr", "", regex=False)
                .str.replace("ty", "", regex=False)
                .str.replace(".", "", regex=False)
                .str.replace(",", "", regex=False)
                .str.extract(r"(\d+)")
                .astype(float)
                / 1_000_000
               )

raw["Năm đăng ký"] = pd.to_numeric(raw["Năm đăng ký"], errors="coerce")
raw["Số Km đã đi"] = (
    raw["Số Km đã đi"].astype(str).str.replace("km", "", regex=False)
    .str.replace(",", "", regex=False).str.replace(".", "", regex=False)
)
raw["Số Km đã đi"] = pd.to_numeric(raw["Số Km đã đi"], errors="coerce")

# Lựa chọn đặc trưng cho modeling
features = raw[[
    "Giá", "Khoảng giá min", "Khoảng giá max", "Năm đăng ký", "Số Km đã đi",
    "Thương hiệu", "Dòng xe", "Loại xe", "Dung tích xe", "Tình trạng"
]].copy()
features = features.dropna(subset=["Giá", "Thương hiệu", "Loại xe"])

numeric_columns = ["Giá", "Khoảng giá min", "Khoảng giá max", "Năm đăng ký", "Số Km đã đi"]
categorical_columns = ["Thương hiệu", "Dòng xe", "Loại xe", "Dung tích xe", "Tình trạng"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_columns),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_columns),
    ]
)

preprocessed = preprocessor.fit_transform(features)
preprocessed


<7203x249 sparse matrix of type '<class 'numpy.float64'>'
	with 72030 stored elements in Compressed Sparse Row format>